In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch

In [4]:
data = pd.read_csv("ChnSentiCorp_htl_all.csv")
data.head()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [5]:
data = data.dropna()

In [6]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv('./ChnSentiCorp_htl_all.csv')
        self.data = self.data.dropna()
    
    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [8]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [37]:
from torch.utils.data import random_split
train_set, valid_set = random_split(dataset, lengths=[0.9, 0.1])
len(train_set), len(valid_set)

(6989, 776)

In [38]:
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")
def collate_fn(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [39]:
from torch.utils.data import DataLoader 
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False, collate_fn=collate_fn)

In [40]:
next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101,  123, 3299,  ..., 3300, 4638,  102],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 6821, 2157,  ...,    0,    0,    0],
        ...,
        [ 101,  122,  510,  ...,    0,    0,    0],
        [ 101, 2791, 7313,  ...,    0,    0,    0],
        [ 101, 2769,  812,  ..., 3198,  671,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 0, 1, 0, 1, 0, 1, 0])}

In [41]:
from torch.optim import Adam
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")
optimizer = Adam(model.parameters(), lr=2e-5)
if torch.cuda.is_available():
    model = model.cuda()

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3

In [42]:
def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f'ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}')
            global_step += 1
        acc = evaluate()
        print(f'ep: {ep}, acc: {acc}')

def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in valid_loader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred =  torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(valid_set)

In [43]:
train()

ep: 0, global_step: 0, loss: 0.646089494228363
ep: 0, global_step: 100, loss: 0.5131412744522095
ep: 0, global_step: 200, loss: 0.27881866693496704
ep: 0, acc: 0.8853092789649963
ep: 1, global_step: 300, loss: 0.2991323471069336
ep: 1, global_step: 400, loss: 0.42628973722457886
ep: 1, acc: 0.8878865838050842
ep: 2, global_step: 500, loss: 0.3797277808189392
ep: 2, global_step: 600, loss: 0.23804496228694916
ep: 2, acc: 0.8878865838050842


In [51]:
sentence = '我觉得这家酒店不错，饭很好吃'
id2_label = {0: "neg", 1: "pos"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sentence, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f'输入: {sentence}\n模型预测结果: {id2_label.get(pred.item())}')

输入: 我觉得这家酒店不错，饭很好吃
模型预测结果: pos


In [52]:
from transformers import pipeline
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer, device=0)
pipe(sentence)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'LABEL_1', 'score': 0.9931585192680359}]